## Logo frame count and duration

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from itertools import groupby
import itertools

In [2]:
inp_df = pd.read_excel('./Microsoft Ad Optimization-AI Data_Survey Aggregated Data 29052019.xlsx',sheet_name="Sheet2")

In [90]:
inp_df['Video Name'].unique()

array(['Yasmin Touch - ATL Win 10 15 TV US',
       'Yasmin Rev - ATL Win 10 30 TV US',
       'Yasmin Power - ATL Win 10 15 TV US',
       'Yasmin - ATL Win 10 30 TV US',
       'Trey Versatility - Surface SP4 15 TV US',
       'Victoria - ATL SP4 30 TV US',
       'Victoria Versatility - ATL SP4 15 TV US',
       'Trey Power - Surface SP4 15 TV US',
       'Tim Flach Power - Surface 15 OLV US',
       'Trey - Surface SP4 30 TV US',
       'Song for Surface NFL Fans - Surface SP4 30 TV US',
       'Surface - Surface SP4 30 TV US',
       'The One Hero - SW Win 10 30 TV US',
       'News Go fast - Surface SP4 30 TV US',
       'Rhythm - Surface Book 30 TV US', 'Shame - Surface SP4 30 TV US',
       'Quads - ATL Win 10 30 TV US', 'Quads Hello - ATL Win 10 15 TV US',
       'Quads Pen - ATL Win 10 15 TV US',
       'Bug Chicks Inking - SW Win 10 15 OLV US',
       'Bug Chicks Inking - SW Win 10 15 TV US',
       'Bug Chicks Inking No Supers - SW Win 10 15 TV US',
       'Bug Chicks Corta

In [3]:
inp_df.columns.values

array(['Video Name', 'Campaign', 'Length of the Ad', 'Country',
       'AD Type: TV/DIGITAL', 'Image Name', 'Age-Gender',
       'Microsoft-Logo', 'Microsoft-Text', 'Intel_Logo', 'Intel-text',
       'Other-logo', 'Other-text', 'Sub-Brands', 'Emotion', 'Ethnicity',
       'Object Class', 'Scene Attributes', 'Object 1-Laptop',
       'object 2- Tablet', 'object 3-Mobile', 'object 4-PC',
       'Interest Trace', 'Microsoft_Logo', 'Microsoft_Windows_Brand',
       'Microsoft_Windows10_Brand', 'Microsoft_SurfacePro4_Brand',
       'Microsoft_SurfaceBook_Brand', 'Microsoft_SurfacePen_Brand',
       'HP_Logo', 'HP_Brand', 'HP_Spectre_Brand', 'HP_Spectrex360_Brand',
       'Apple_Logo', 'Apple_MacbookAir_Brand', 'Apple_MacBookAir13_Brand',
       'Apple_MacBookPro_Brand', 'Intel_Logo.1', 'Dell_Logo',
       'Dell_Brand', 'Dell_XPS13_Brand', 'Asus_Logo', 'Asus_Brand',
       'Asus_T102_Brand', 'Lenovo_Brand', 'Lenovo_Yoga_Brand',
       'Lenovo_Yoga910_Brand', 'No. of Microsoft mention in Audi

In [44]:
ethinicity = inp_df[['Ethnicity','Video Name']].groupby(['Ethnicity','Video Name']).size().unstack().unstack()
emotion = inp_df[['Emotion','Video Name']].groupby(['Emotion','Video Name']).size().unstack().unstack()

In [45]:
inputs_df = inp_df[['Country','Video Name']].groupby(['Video Name','Country']).size().unstack()
inputs_df['Video Name'] = inputs_df.index.values

In [86]:
temp_df = inp_df[['Length of the Ad','Video Name']].groupby(['Video Name','Length of the Ad']).size().unstack()
inputs_df['length_of_ad_15'] = temp_df[temp_df.columns[0]].values
inputs_df.loc[inputs_df.length_of_ad_15.isnull(),'length_of_ad_15'] = 0
inputs_df.loc[inputs_df.length_of_ad_15 != 0,'length_of_ad_15'] = 1

In [88]:
temp_df = inp_df[['AD Type: TV/DIGITAL','Video Name']].groupby(['Video Name','AD Type: TV/DIGITAL']).size().unstack()
inputs_df['ad_type_tv'] = temp_df[temp_df.columns[1]].values
inputs_df.loc[inputs_df.ad_type_tv.isnull(),'ad_type_tv'] = 0
inputs_df.loc[inputs_df.ad_type_tv != 0,'ad_type_tv'] = 1

In [48]:
inp_df['Age'] = inp_df['Age-Gender']
inp_df['Gender'] = inp_df['Age-Gender']
for i in range(len(inp_df['Age-Gender'])):
    if len(inp_df['Age'][i].split(',')) == 2:
        inp_df.loc[i,'Age'] = inp_df.loc[i,'Age-Gender'].split(',')[0]
        inp_df.loc[i,'Age'] = (int(inp_df.Age[i].split(":")[1].split("-")[0]) + int(inp_df.Age[i].split(":")[1].split("-")[1].replace("'","")))/2
        inp_df.loc[i,'Gender'] = inp_df.loc[i,'Age-Gender'].split(',')[1]
    else:
        inp_df.loc[i,'Age'] = float('nan')
        
inp_df.Age = inp_df.Age.astype('float')
age_df = inp_df[['Video Name','Age']]
age_df_vid = inp_df[['Video Name','Age']].groupby(['Video Name','Age']).size().unstack()

In [49]:
inputs_df = inputs_df.set_index(inputs_df['Video Name'])

In [50]:
inputs_df = pd.concat((inputs_df,age_df_vid),axis=1)

/home/internbmilap562/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [51]:
inputs_df[['Age: 0-5_frame_count','Age: 5-15_frame_count','Age: 15-24_frame_count','Age: 25-34_frame_count','Age: 35-44_frame_count','Age: 45-60_frame_count']]=inputs_df[[2.5,7.5,19.5,29.5,39.5,52.5]]

In [52]:
inputs_df = inputs_df.drop([2.5,7.5,19.5,29.5,39.5,52.5],1)

In [53]:
temp_df = inp_df[['Gender','Video Name']].groupby(['Video Name','Gender']).size().unstack(fill_value =0).drop('No Face Found',1)

In [54]:
def lower(row,col):
    row[col] = row[col].lower()
    return row

In [55]:
def add(x):
    return x[0]+x[1]

In [56]:
def agg_logo_text_data(columns_):
    logo_df = pd.DataFrame()
    for c in columns_:
        print(c)
        if c=='Microsoft-Logo':
            inp_df[inp_df[c].isna()==False] = inp_df[inp_df[c].isna()==False].apply(lambda x: lower(x,c),1)  
        temp_logo_df = inp_df[[c,'Video Name']].groupby(['Video Name',c]).size().unstack(fill_value = 0)
        #else:
        #    logo_df = pd.concat([logo_df,inp_df[[c,'Video Name']].groupby(['Video Name',c]).size().unstack(fill_value = 0)],axis=1)

        ## Duration
        dur_df = temp_logo_df*0.5
        var_cols =temp_logo_df.columns.values
        #print(var_cols)
        dur_df.columns = temp_logo_df.columns.values + '_dur'
        
        if len(logo_df) == 0:
            logo_df = pd.concat([temp_logo_df,dur_df],axis=1)
        else:
            frame_dur_df = pd.concat([temp_logo_df,dur_df],axis=1)
            logo_df = pd.concat([logo_df,frame_dur_df],axis=1)

        ## Occurences


        vids = temp_logo_df.index
    #     for unq_val in range(len(inp_df.loc[:,c].unique())):

    #         if str(inp_df.loc[:,c].unique()[unq_val]) != 'nan':
    #             #print(unq_val)
        unq_val = 1
        ocr_list = list(itertools.product(var_cols,['_first_ocr','_last_ocr','_total_ocr'])) 
        ocr_cols = map(add,ocr_list)
        ocr_df = pd.DataFrame(columns = list(ocr_cols))
        for v in range(len(vids)):
    #                 print(inp_df.loc[:,c].unique()[unq_val])
    #                 print(len(inp_df.loc[inp_df['Video Name'] == vids[v],c].unique()))
            first_ocr =[None]*len(var_cols)
            last_ocr =[None]*len(var_cols)
            total_ocr =[None]*len(var_cols)

            if len(inp_df.loc[inp_df['Video Name'] == vids[v],c].unique()) >1:
                #print(inp_df.loc[inp_df['Video Name'] == vids[v],c].unique())
                for k,var in enumerate(var_cols):
                    ocr_op =   [i for i,x in enumerate(inp_df.loc[inp_df['Video Name'] == vids[v],c]) if x==var]
    #                 print(ocr_op)

                    if len(ocr_op) > 0:
                        ocr_df.loc[v,str(var_cols[k]+'_first_ocr')] = ocr_op[0]*0.5
                        ocr_df.loc[v,str(var_cols[k]+'_last_ocr')] = ocr_op[-1]*0.5
                        ocr_df.loc[v,str(var_cols[k]+'_total_ocr')] = len([x for x in list(np.diff(ocr_op)) if x != 1]) +1
                    else:
                        ocr_df.loc[v,str(var_cols[k]+'_first_ocr')] = None
                        ocr_df.loc[v,str(var_cols[k]+'_last_ocr')] = None
                        ocr_df.loc[v,str(var_cols[k]+'_total_ocr')] = None

            else:
                ocr_df.loc[v,str(var_cols[0]+'_first_ocr')] = None
                ocr_df.loc[v,str(var_cols[0]+'_last_ocr')] = None
                ocr_df.loc[v,str(var_cols[0]+'_total_ocr')] = None
        ocr_df.index = vids
        logo_df = pd.concat([logo_df,ocr_df],axis=1)
            #print(logo_df)
    return logo_df

In [57]:
logo_df = agg_logo_text_data(['Microsoft-Logo','Intel_Logo','Sub-Brands','Other-text'])

Microsoft-Logo
Intel_Logo
Sub-Brands


/home/internbmilap562/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/internbmilap562/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Other-text


In [58]:
agg_inp = pd.DataFrame()
agg_inp['Video Name'] = inputs_df.index.values
inputs_df = inputs_df.set_index(agg_inp.index.values)
agg_inp[['Country_AU','Country_US','Country_UK']] = inputs_df[['AU','US','UK']]

In [59]:
ethinicity = ethinicity.unstack()[['African','Caucasian','Eastasian','Latino']].set_index(agg_inp.index.values)
emotion = emotion.unstack()[['Angry','Disgust','Fear','Happiness','Neutral','Sadness','Surprise']].set_index(agg_inp.index.values)

In [60]:
agg_inp[['African_frame_count','Caucasian_frame_count','Eastasian_frame_count','Latino_frame_count']] = ethinicity[['African','Caucasian','Eastasian','Latino']]
agg_inp[['Angry_frame_count','Disgust_frame_count','Fear_frame_count','Happiness_frame_count','Neutral_frame_count','Sadness_frame_count','Surprise_frame_count']] = emotion[['Angry','Disgust','Fear','Happiness','Neutral','Sadness','Surprise']]

In [61]:
agg_inp[['length_of_ad_15','ad_type_tv','Age: 0-5_frame_count','Age: 5-15_frame_count','Age: 15-24_frame_count','Age: 25-34_frame_count','Age: 35-44_frame_count','Age: 45-60_frame_count']] = inputs_df[['length_of_ad_15','ad_type_tv','Age: 0-5_frame_count','Age: 5-15_frame_count','Age: 15-24_frame_count','Age: 25-34_frame_count','Age: 35-44_frame_count','Age: 45-60_frame_count']]

In [62]:
temp_df = temp_df.set_index(agg_inp.index.values)

In [63]:
agg_inp[["Female_frame_count","Male_frame_count"]] = temp_df[[" 'Female']"," 'Male']"]]

In [64]:
# logo_df.index.values == agg_inp['Video Name']

In [65]:
# logo_df['Video Name'] = logo_df.index.values
# logo_df = logo_df.drop('Video Name',1)

In [66]:
agg_inp = agg_inp.set_index(agg_inp['Video Name'])

In [67]:
agg_inp = pd.concat((agg_inp,logo_df),axis=1)

/home/internbmilap562/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [68]:
def countryEncoding(x):
    for c in ['Country_AU','Country_US','Country_UK']:
        if x[c]>0:
            x[c] = 1
        else:
            x[c] =0
    return x

In [69]:
agg_inp = agg_inp.apply(countryEncoding,1)

In [70]:
frame_col = [col for col in agg_inp.columns if '_frame_count' in col]

In [71]:
for col in frame_col:
    index = col.find('_frame_count')
    new_col = col[:index]+'_dur'
    agg_inp[new_col] = agg_inp[col]*0.5

In [72]:
total_cols =['Video Name', 'Country_AU', 'Country_UK', 'Country_US',
       'length_of_ad_15', 'ad_type_tv', 'Female_frame_count',
       'Male_frame_count', 'Female_dur', 'Male_dur',
       'Age: 5-15_frame_count', 'Age: 0-5_frame_count',
       'Age: 15-24_frame_count', 'Age: 25-34_frame_count',
       'Age: 35-44_frame_count', 'Age: 45-60_frame_count',
       'Age: 5-15_dur', 'Age: 0-5_dur', 'Age: 15-24_dur',
       'Age: 25-34_dur', 'Age: 35-44_dur', 'Age: 45-60_dur',
       'Angry_frame_count', 'Disgust_frame_count', 'Fear_frame_count',
       'Happiness_frame_count', 'Neutral_frame_count',
       'Sadness_frame_count', 'Surprise_frame_count', 'Angry_dur',
       'Disgust_dur', 'Fear_dur', 'Happiness_dur', 'Neutral_dur',
       'Sadness_dur', 'Surprise_dur', 'African_frame_count',
       'Caucasian_frame_count', 'Eastasian_frame_count',
       'Latino_frame_count', 'African_dur', 'Caucasian_dur',
       'Eastasian_dur', 'Latino_dur', 'microsoft', 'microsoft_dur',
       'microsoft_first_ocr', 'microsoft_last_ocr', 'microsoft_total_ocr',
       'Intel_dur', 'Intel_first_ocr', 'Intel_last_ocr',
       'Intel_total_ocr', 'ASUS T102', 'Dell PCs', 'Dell XPS',
       'Dell XPS 13', 'HP Spectre x360', 'HP Spectre x360, Windows',
       'Lenovo PCs', 'Lenovo Yoga 720', 'Lenovo Yoga 910', 'MacBook Air',
       'MacBook Pro', 'Macbook Air', 'Microsoft Surface', 'Suface Laptop',
       'Surface Book', 'Surface Go', 'Surface Laptop', 'Surface Pen',
       'Surface Pro', 'Surface Pro 4',
       'Surface Pro 4, Apple_MacBookAir13',
       'Surface Pro 4, HP Spectre x360', 'Surface Pro 4, MacBook Air',
       'Surface Pro 4, MacBook Air13, MacBook Air',
       'Surface Pro 4, Surface Book', 'Surface Pro 4, Windows 10',
       'Surface Pro 6', 'Surface laptop', 'Surface pro 4',
       'The new Surface Pro', 'The surface Laptop', 'Windows',
       'Windows 10', 'Windows 10 PC', 'Windows 10 PCs',
       'Windows 10, HP_Spectre', 'Windows 10, Windows', 'Yoga',
       'ASUS T102_dur', 'Dell PCs_dur', 'Dell XPS_dur', 'Dell XPS 13_dur',
       'HP Spectre x360_dur', 'HP Spectre x360, Windows_dur',
       'Lenovo PCs_dur', 'Lenovo Yoga 720_dur', 'Lenovo Yoga 910_dur',
       'MacBook Air_dur', 'MacBook Pro_dur', 'Macbook Air_dur',
       'Microsoft Surface_dur', 'Suface Laptop_dur', 'Surface Book_dur',
       'Surface Go_dur', 'Surface Laptop_dur', 'Surface Pen_dur',
       'Surface Pro_dur', 'Surface Pro 4_dur',
       'Surface Pro 4, Apple_MacBookAir13_dur',
       'Surface Pro 4, HP Spectre x360_dur',
       'Surface Pro 4, MacBook Air_dur',
       'Surface Pro 4, MacBook Air13, MacBook Air_dur',
       'Surface Pro 4, Surface Book_dur', 'Surface Pro 4, Windows 10_dur',
       'Surface Pro 6_dur', 'Surface laptop_dur', 'Surface pro 4_dur',
       'The new Surface Pro_dur', 'The surface Laptop_dur', 'Windows_dur',
       'Windows 10_dur', 'Windows 10 PC_dur', 'Windows 10 PCs_dur',
       'Windows 10, HP_Spectre_dur', 'Windows 10, Windows_dur',
       'Yoga_dur', 'Windows 10_first_ocr', 'Windows 10_last_ocr',
       'Windows 10_total_ocr', 'Lenovo Yoga 910_first_ocr',
       'Lenovo Yoga 910_last_ocr', 'Lenovo Yoga 910_total_ocr',
       'Yoga_first_ocr', 'Yoga_last_ocr', 'Yoga_total_ocr',
       'Surface Pro 4_first_ocr', 'Surface Pro 4_last_ocr',
       'Surface Pro 4_total_ocr',
       'Surface Pro 4, Apple_MacBookAir13_first_ocr',
       'Surface Pro 4, Apple_MacBookAir13_last_ocr',
       'Surface Pro 4, Apple_MacBookAir13_total_ocr',
       'Surface Pro 4, MacBook Air13, MacBook Air_first_ocr',
       'Surface Pro 4, MacBook Air13, MacBook Air_last_ocr',
       'Surface Pro 4, MacBook Air13, MacBook Air_total_ocr',
       'Surface Book_first_ocr', 'Surface Book_last_ocr',
       'Surface Book_total_ocr', 'MacBook Pro_first_ocr',
       'MacBook Pro_last_ocr', 'MacBook Pro_total_ocr',
       'Microsoft Surface_first_ocr', 'Microsoft Surface_last_ocr',
       'Microsoft Surface_total_ocr', 'MacBook Air_first_ocr',
       'MacBook Air_last_ocr', 'MacBook Air_total_ocr',
       'Surface Pro 4, Surface Book_first_ocr',
       'Surface Pro 4, Surface Book_last_ocr',
       'Surface Pro 4, Surface Book_total_ocr', 'ASUS T102_first_ocr',
       'ASUS T102_last_ocr', 'ASUS T102_total_ocr',
       'Windows 10, HP_Spectre_first_ocr',
       'Windows 10, HP_Spectre_last_ocr',
       'Windows 10, HP_Spectre_total_ocr', 'HP Spectre x360_first_ocr',
       'HP Spectre x360_last_ocr', 'HP Spectre x360_total_ocr',
       'Surface Pro 4, Windows 10_first_ocr',
       'Surface Pro 4, Windows 10_last_ocr',
       'Surface Pro 4, Windows 10_total_ocr',
       'Surface Pro 4, HP Spectre x360_first_ocr',
       'Surface Pro 4, HP Spectre x360_last_ocr',
       'Surface Pro 4, HP Spectre x360_total_ocr',
       'Windows 10, Windows_first_ocr', 'Windows 10, Windows_last_ocr',
       'Windows 10, Windows_total_ocr', 'Windows_first_ocr',
       'Windows_last_ocr', 'Windows_total_ocr',
       'HP Spectre x360, Windows_first_ocr',
       'HP Spectre x360, Windows_last_ocr',
       'HP Spectre x360, Windows_total_ocr', 'Dell XPS 13_first_ocr',
       'Dell XPS 13_last_ocr', 'Dell XPS 13_total_ocr',
       'Surface Pen_first_ocr', 'Surface Pen_last_ocr',
       'Surface Pen_total_ocr', 'Macbook Air_first_ocr',
       'Macbook Air_last_ocr', 'Macbook Air_total_ocr',
       'Windows 10 PC_first_ocr', 'Windows 10 PC_last_ocr',
       'Windows 10 PC_total_ocr', 'Windows 10 PCs_first_ocr',
       'Windows 10 PCs_last_ocr', 'Windows 10 PCs_total_ocr',
       'Surface Pro 4, MacBook Air_first_ocr',
       'Surface Pro 4, MacBook Air_last_ocr',
       'Surface Pro 4, MacBook Air_total_ocr', 'Surface pro 4_first_ocr',
       'Surface pro 4_last_ocr', 'Surface pro 4_total_ocr',
       'Lenovo Yoga 720_first_ocr', 'Lenovo Yoga 720_last_ocr',
       'Lenovo Yoga 720_total_ocr', 'Dell PCs_first_ocr',
       'Dell PCs_last_ocr', 'Dell PCs_total_ocr',
       'Surface Laptop_first_ocr', 'Surface Laptop_last_ocr',
       'Surface Laptop_total_ocr', 'The new Surface Pro_first_ocr',
       'The new Surface Pro_last_ocr', 'The new Surface Pro_total_ocr',
       'Surface Pro_first_ocr', 'Surface Pro_last_ocr',
       'Surface Pro_total_ocr', 'Suface Laptop_first_ocr',
       'Suface Laptop_last_ocr', 'Suface Laptop_total_ocr',
       'Surface laptop_first_ocr', 'Surface laptop_last_ocr',
       'Surface laptop_total_ocr', 'The surface Laptop_first_ocr',
       'The surface Laptop_last_ocr', 'The surface Laptop_total_ocr',
       'Surface Go_first_ocr', 'Surface Go_last_ocr',
       'Surface Go_total_ocr', 'Surface Pro 6_first_ocr',
       'Surface Pro 6_last_ocr', 'Surface Pro 6_total_ocr',
       'Dell XPS_first_ocr', 'Dell XPS_last_ocr', 'Dell XPS_total_ocr',
       'Lenovo PCs_first_ocr', 'Lenovo PCs_last_ocr',
       'Lenovo PCs_total_ocr']

In [77]:
i=0;
for c in agg_inp.columns.values:
    if c in total_cols:
        i+=1
    else:
        print(c)
print(i)

Intel
Asus
Currys PCs world
Dell
Hp
Lenovo
Asus_dur
Currys PCs world_dur
Dell_dur
Hp_dur
Lenovo_dur
Asus_first_ocr
Asus_last_ocr
Asus_total_ocr
Currys PCs world_first_ocr
Currys PCs world_last_ocr
Currys PCs world_total_ocr
Dell_first_ocr
Dell_last_ocr
Dell_total_ocr
Hp_first_ocr
Hp_last_ocr
Hp_total_ocr
Lenovo_first_ocr
Lenovo_last_ocr
Lenovo_total_ocr
248
